# Benchmarking with Argo Worfklows & Vegeta

In this notebook we will dive into how you can run bench marking with batch processing with Argo Workflows, Seldon Core and Vegeta.

Dependencies:

* Seldon core installed as per the docs with Istio as an ingress 
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Setup

### Install Seldon Core
Use the notebook to [set-up Seldon Core with Ambassador or Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

Note: If running with KIND you need to make sure do follow [these steps](https://github.com/argoproj/argo-workflows/issues/2376#issuecomment-595593237) as workaround to the `/.../docker.sock` known issue.


### Install Argo Workflows
You can follow the instructions from the official [Argo Workflows Documentation](https://github.com/argoproj/argo#quickstart).

Download the right CLi for your environment following the documentation (https://github.com/argoproj/argo-workflows/releases/tag/v3.0.8)

You also need to make sure that argo has permissions to create seldon deployments - for this you can just create a default-admin rolebinding as follows:

Set up the RBAC so the argo workflow is able to create seldon deployments.

Set up the configmap in order for it to work in KIND and other environments where Docker may not be thr main runtime (see https://github.com/argoproj/argo-workflows/issues/5243#issuecomment-792993742)

### Create Benchmark Argo Workflow

In order to create a benchmark, we created a simple argo workflow template so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 1 replicas and 4 cpus (with 100 max workers) to send requests.

In [7]:
!helm template seldon-benchmark-workflow ../../../helm-charts/seldon-benchmark-workflow/ \
    --set workflow.namespace=argo \
    --set workflow.name=seldon-benchmark-process \
    --set workflow.parallelism=2 \
    --set seldonDeployment.name=sklearn \
    --set seldonDeployment.replicas="1" \
    --set seldonDeployment.serverWorkers="5" \
    --set seldonDeployment.serverThreads=1 \
    --set seldonDeployment.modelUri="gs://seldon-models/v1.10.0-dev/sklearn/iris" \
    --set seldonDeployment.server="SKLEARN_SERVER" \
    --set seldonDeployment.apiType="rest|grpc" \
    --set seldonDeployment.requests.cpu="2000Mi" \
    --set seldonDeployment.limits.cpu="2000Mi" \
    --set seldonDeployment.disableOrchestrator="true|false" \
    --set benchmark.cpu="2" \
    --set benchmark.concurrency="1" \
    --set benchmark.duration="30s" \
    --set benchmark.rate=0 \
    --set benchmark.data='\{"data": {"ndarray": [[0\,1\,2\,3]]\}\}' \
    | argo submit -

Name:                seldon-benchmark-process
Namespace:           argo
ServiceAccount:      default
Status:              Pending
Created:             Mon Jun 28 18:38:12 +0100 (now)
Progress:            


In [7]:
!argo list -n argo

NAME                       STATUS    AGE   DURATION   PRIORITY
seldon-benchmark-process   Running   20s   20s        0


In [8]:
!argo logs -f seldon-benchmark-process -n argo

seldon-benchmark-process-635956972: [{"name": "sklearn-0", "replicas": "1", "serverWorkers": "5", "serverThreads": "1", "modelUri": "gs://seldon-models/sklearn/iris", "image": "", "server": "SKLEARN_SERVER", "apiType": "rest", "requestsCpu": "2000Mi", "requestsMemory": "100Mi", "limitsCpu": "2000Mi", "limitsMemory": "1000Mi", "benchmarkCpu": "2", "concurrency": "1", "duration": "30s", "rate": "0", "disableOrchestrator": "true", "params": "{\"name\": \"sklearn-0\", \"replicas\": \"1\", \"serverWorkers\": \"5\", \"serverThreads\": \"1\", \"modelUri\": \"gs://seldon-models/sklearn/iris\", \"image\": \"\", \"server\": \"SKLEARN_SERVER\", \"apiType\": \"rest\", \"requestsCpu\": \"2000Mi\", \"requestsMemory\": \"100Mi\", \"limitsCpu\": \"2000Mi\", \"limitsMemory\": \"1000Mi\", \"benchmarkCpu\": \"2\", \"concurrency\": \"1\", \"duration\": \"30s\", \"rate\": \"0\", \"disableOrchestrator\": \"true\"}"}]
seldon-benchmark-process-2323867814: time="2021-06-28T17:27:52.287Z" level=info msg="Starti

In [14]:
!argo get seldon-benchmark-process -n argo

Name:                seldon-benchmark-process
Namespace:           argo
ServiceAccount:      default
Status:              Running
Conditions:          
 PodRunning          False
Created:             Mon Jun 28 18:38:12 +0100 (4 minutes ago)
Started:             Mon Jun 28 18:38:12 +0100 (4 minutes ago)
Duration:            4 minutes 53 seconds
Progress:            14/15
ResourcesDuration:   4m41s*(1 cpu),4m41s*(100Mi memory)

STEP                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## Process the results

We can now print the results in a consumable format.

## Deeper Analysis
Now that we have all the parameters, we can do a deeper analysis

True

In [ ]:
import sys

sys.path.append("../../../testing/scripts")
import pandas as pd
from seldon_e2e_utils import bench_results_from_output_logs

results = bench_results_from_output_logs("seldon-benchmark-process", namespace="argo")
df = pd.DataFrame.from_dict(results)

In [16]:
df.head()

,mean,50th,90th,95th,99th,throughputAchieved,success,errors,name,replicas,...,apiType,requestsCpu,requestsMemory,limitsCpu,limitsMemory,benchmarkCpu,concurrency,duration,rate,disableOrchestrator
0,4.573302,4.018635,6.225710,7.480878,13.893386,218.518811,6559,0,sklearn-0,1,...,rest,2000Mi,100Mi,2000Mi,1000Mi,2,1,30s,0,true
1,4.565145,3.939032,6.785393,7.928704,13.315820,218.892806,6568,0,sklearn-1,1,...,rest,2000Mi,100Mi,2000Mi,1000Mi,2,1,30s,0,false
2,3.747319,3.212300,5.651600,6.858700,9.191800,258.595746,7757,1,sklearn-2,1,...,grpc,2000Mi,100Mi,2000Mi,1000Mi,2,1,30s,0,true
3,4.271879,3.855800,6.495800,7.353500,8.980500,226.930063,6807,1,sklearn-3,1,...,grpc,2000Mi,100Mi,2000Mi,1000Mi,2,1,30s,0,false


In [51]:
!argo delete seldon-benchmark-process -n argo || echo "Argo workflow already deleted or not exists"

Workflow 'seldon-benchmark-process' deleted
